## Note - This example code uses a SQL Server instance running on a docker Linux container.  If you have created a container using the BuildContainer notebook, use these scripts as is.  If you are running on a native instance, modify the scripts according the comments.  

## Step 1 - Connect to your SQL Server instance using Azure Data Studio.

In [9]:
/* If you created a SQL container in the previous step, you can use this script as is.
   If you are using a different instance, change the FILENAME paths to be valid.
   Change the database and object names if you must.  */
USE master
GO
DROP DATABASE IF EXISTS whitesox
GO
CREATE DATABASE whitesox
ON PRIMARY
(NAME = N'whitesox_primary', FILENAME = '/var/opt/mssql/data/whitesox.mdf', SIZE = 2Gb , MAXSIZE = UNLIMITED, FILEGROWTH = 65536KB)
LOG ON 
(NAME = N'whitesox_Log', FILENAME = '/var/opt/mssql/data/whitesox_log.ldf', SIZE = 5Gb , MAXSIZE = UNLIMITED , FILEGROWTH = 65536KB)
GO
ALTER DATABASE whitesox SET RECOVERY SIMPLE
GO
USE whitesox
GO
DROP TABLE IF EXISTS letsgowhitesox
GO
CREATE TABLE letsgowhitesox (playerid int primary key clustered, playername char(7000) not null)
GO
SET NOCOUNT ON
GO
BEGIN TRAN
DECLARE @x int
SET @x = 0
WHILE (@x < 100000)
BEGIN
	INSERT INTO letsgowhitesox VALUES (@x, 'Hoping for baseball with fans next season!')
	SET @x = @x + 1
END
COMMIT TRAN
GO
SET NOCOUNT OFF

USE master;

SELECT name, is_accelerated_database_recovery_on
FROM sys.databases
WHERE name = N'whitesox';



Query was canceled by user

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

...

Commands completed successfully.

Commands completed successfully.

(1 row affected)

Total execution time: 00:00:40.909

name,is_accelerated_database_recovery_on
whitesox,0


## Step 2 - Delete all rows for the table in an open transaction.

In [2]:
USE master
GO
ALTER DATABASE whitesox SET ACCELERATED_DATABASE_RECOVERY = OFF
GO
USE whitesox
GO
BEGIN TRAN
DELETE from letsgowhitesox
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

(100000 rows affected)

Total execution time: 00:00:18.463

## Step 3 - Check transaction log usage.
### Because there is an open transaction, the log file will contain space that cannot be reused.  After checking log usage, run a CHECKPOINT and see if log space is freed.

In [3]:
USE whitesox;

SELECT * FROM sys.dm_db_log_space_usage

CHECKPOINT;

SELECT * FROM sys.dm_db_log_space_usage

GO

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.572

database_id,total_log_size_in_bytes,used_log_space_in_bytes,used_log_space_in_percent,log_space_in_bytes_since_last_backup
5,5368700928,4223852544,78.67551,993247232


database_id,total_log_size_in_bytes,used_log_space_in_bytes,used_log_space_in_percent,log_space_in_bytes_since_last_backup
5,5368700928,4223950848,78.67734,1000058880


## Step 4 - How long does it take to rollback the deletes?
### Rollback the transaction and check the duration.

In [4]:
USE whitesox;

ROLLBACK TRAN
GO

Commands completed successfully.

Total execution time: 00:00:30.641

## Step 5 - Enable Accelerated Database Recovery

In [5]:
USE master
GO
ALTER DATABASE whitesox SET ACCELERATED_DATABASE_RECOVERY = ON
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.069

## Step 6 - Delete all of the rows again within a transaction

In [6]:
USE whitesox
GO
BEGIN TRAN
DELETE from letsgowhitesox
GO

Commands completed successfully.

(100000 rows affected)

Total execution time: 00:00:11.496

## Step 7 - Check log space used before and after a CHECKPOINT
### Notice log space usage is low even before the CHECKPOINT

In [7]:
USE whitesox;
SELECT * FROM sys.dm_db_log_space_usage
GO

CHECKPOINT;

SELECT * FROM sys.dm_db_log_space_usage
GO


(1 row affected)

(1 row affected)

Total execution time: 00:00:00.810

database_id,total_log_size_in_bytes,used_log_space_in_bytes,used_log_space_in_percent,log_space_in_bytes_since_last_backup
5,5368700928,1056526336,19.679367,765849600


database_id,total_log_size_in_bytes,used_log_space_in_bytes,used_log_space_in_percent,log_space_in_bytes_since_last_backup
5,5368700928,385732608,7.1848407,159744


## Step 8 - How fast is a ROLLBACK?
### Is ROLLBACK any faster with ADR enabled?

In [8]:
USE whitesox;

ROLLBACK TRAN
GO

Commands completed successfully.

Total execution time: 00:00:00.002